**Tabla de contenido**

- [Examinando el modelo VAR](#Examinando-el-modelo-VAR)
- [Diseñando un procedimiento de modelado para el modelo VAR(p)](#Diseñando-un-procedimiento-de-modelado-para-el-modelo-VA)
    - [Explorando el test de causalidad de Granger](#Explorando-el-test-de-causalidad-de-Granger)

En el último capítulo, viste cómo el modelo SARIMAX se puede utilizar para incluir el impacto de variables exógenas en una serie temporal. Con el modelo SARIMAX, la relación es unidireccional: asumimos que la variable exógena tiene un impacto en el objetivo solamente.

Sin embargo, es posible que dos series temporales tengan una relación bidireccional, lo que significa que la serie temporal t1 es un predictor de la serie temporal t2, y la serie temporal t2 también es un predictor para la serie temporal t1. En tal caso, sería útil tener un modelo que pueda tener en cuenta esta relación bidireccional y generar predicciones para ambas series temporales simultáneamente.

Esto nos lleva al `modelo de autorregresión vectorial (VAR)`. Este modelo en particular nos permite captar la relación entre múltiples series temporales a medida que cambian con el tiempo. Eso, a su vez, nos permite producir pronósticos para muchas series temporales simultáneamente, realizando así pronósticos multivariantes.

# Examinando el modelo VAR

`El modelo de autorregresión vectorial (VAR) captura la relación entre múltiples series a medida que cambian con el tiempo.` En este modelo, cada serie tiene un impacto en las otras, a diferencia del modelo SARIMAX donde la variable exógena tenía un impacto en el objetivo, pero no al revés. Recuerda que en el capítulo 9 usamos las variables realcons, realinv, realgovt, realdpi, cpi, m1 y tbilrate como predictores para realgdp, pero no consideramos cómo realgdp puede afectar a ninguna de esas variables. Esa es la razón por la cual usamos el modelo SARIMAX en ese caso.

Puede que hayas notado el regreso de la autorregresión, lo que nos lleva de vuelta al modelo AR(p) del capítulo 5. Esta es una buena intuición, ya que el modelo VAR se puede ver como una generalización del modelo AR(p) para permitir la previsión de múltiples series temporales. Por lo tanto, también podemos denotar el modelo VAR como VAR(p), donde p es el orden y tiene el mismo significado que en el modelo AR(p).

`Recuerda que AR(p) expresa el valor de una serie temporal como una combinación lineal de una constante` $C$, el término de error presente $ϵ_t$, que también es ruido blanco, y los valores pasados de la serie $y_{t–p}$. La magnitud de la influencia de los valores pasados en el valor presente se denota como $ϕ_p$, que representa los coeficientes del modelo AR(p), como se muestra en la siguiente ecuación.

$$y_t = C + ϕ_1y_{t–1} + ϕ_2y_{t–2} +⋅⋅⋅+ ϕ_py_{t–p} + ϵ_t$$

Podemos simplemente extender la anterior ecuación  para permitir que se modelen múltiples series de tiempo, donde cada una tiene un impacto en las otras.

Para simplificar, consideremos un sistema con dos series temporales, denominadas $y_{1,t}$ y $y_{2,t}$, y un orden de 1, lo que significa que p = 1. Luego, utilizando notación de matrices, el modelo VAR(1) se puede expresar como la siguiente ecuación.
$$
\begin{bmatrix}
y_{1,t} \\
y_{2,t}
\end{bmatrix}
=
\begin{bmatrix}
C_1 \\
C_2
\end{bmatrix}
+
\begin{bmatrix}
\phi_{1,1} & \phi_{1,2} \\
\phi_{2,1} & \phi_{2,2}
\end{bmatrix}
\begin{bmatrix}
y_{1,t-1} \\
y_{2,t-1}
\end{bmatrix}
+
\begin{bmatrix}
\epsilon_{1,t} \\
\epsilon_{2,t}
\end{bmatrix}
$$

Al realizar la multiplicación de matrices, tenemos:

$y_{1,t} = C_1 + ϕ_{1,1}y_{1,t–1} + ϕ_{1,2}y_{2,t–1} + ϵ_{1,t}$

$y_{2,t} = C_2 + ϕ_{2,1}y_{1,t–1} + ϕ_{2,2}y_{2,t–1} + ϵ_{2,t}$

El modelo VAR(p) solo es válido si ambas series son útiles para predecirse mutuamente. Observar la forma general de las series a lo largo del tiempo no es suficiente para respaldar esa hipótesis. En cambio, debemos aplicar `la prueba de causalidad de Granger, que es una prueba de hipótesis estadística para determinar si una serie temporal es predictiva de otra`. Solo tras el éxito de esta prueba podemos aplicar el modelo VAR para hacer predicciones. Este es un paso importante en nuestro procedimiento de modelado al utilizar un modelo VAR.

# Diseñando un procedimiento de modelado para el modelo VAR

El modelo VAR(p) requiere una versión ligeramente modificada del procedimiento de modelado que hemos estado utilizando. La modificación más notable es la adición de la prueba de causalidad de Granger, ya que el modelo VAR asume que los valores pasados de ambas series temporales son significativamente predictivos de la otra serie temporal.

![VAR(p)](fig/VAR.png)

Figura. Procedimiento de modelado para el modelo VAR(p). Es muy similar a los procedimientos de modelado que hemos estado usando desde la introducción del modelo ARMA(p, q), pero esta vez estamos ajustando diferentes modelos VAR(p) y seleccionando el que tiene el menor AIC. Luego realizamos la prueba de causalidad de Granger. Si falla, el modelo VAR(p) no es válido y no continuaremos con el procedimiento. Por otro lado, si la prueba pasa, realizamos un análisis de residuos. Si los residuos son similares al ruido blanco, el modelo VAR(p) se puede utilizar para pronósticos.


`La principal diferencia aquí es que solo enumeramos valores para el orden p`, ya que estamos ajustando diferentes modelos VAR(p) a los datos. Luego, una vez que se ha seleccionado el modelo con el AIC más bajo, realizamos la prueba de causalidad de Granger. Esta prueba determina si los valores pasados de una serie temporal son estadísticamente significativos para pronosticar otra serie temporal. Es importante verificar esta relación porque el modelo VAR(p) utiliza valores pasados de una serie temporal para pronosticar otra.

`Si la prueba de causalidad de Granger falla, no podemos decir que los valores pasados de una serie temporal sean predictivos de la otra serie temporal`. En ese caso, el modelo VAR(p) se vuelve inválido y debemos recurrir a una variación del modelo SARIMAX para pronosticar la serie temporal. `Por otro lado, si la prueba de causalidad de Granger pasa, podemos reanudar el procedimiento con el análisis de residuos`. Como antes, si los residuos están cerca del ruido blanco, podemos usar el modelo VAR(p) seleccionado para hacer pronósticos.

Antes de pasar a aplicar este procedimiento de modelado, vale la pena dedicar un tiempo a explorar la prueba de causalidad de Granger con más detalle.

## Explorando el test de causalidad de Granger

`Como se muestra en la sección anterior, el modelo VAR(p) asume que cada serie temporal tiene un impacto en otra.` Por lo tanto, es importante probar si esta relación realmente existe. De lo contrario, estaríamos asumiendo una relación que no existe, lo que introduciría errores en el modelo y haría que nuestras predicciones fueran inválidas e inciertas.

Por lo tanto, utilizamos la prueba de causalidad de Granger. Esta es una prueba estadística que nos ayuda a determinar si los valores pasados de una serie temporal $y_{2,t}$ pueden ayudar a predecir la serie temporal $y_{1,t}$. Si ese es el caso, entonces decimos que $y_{2,t}$ causa a $y_{1,t}$ según Granger.

`Tenga en cuenta que la prueba de causalidad de Granger está restringida a la causalidad predictiva`, ya que solo estamos determinando si los valores pasados de una serie temporal son estadísticamente significativos para predecir otra serie temporal. Además, la prueba requiere que ambas series temporales sean estacionarias para que los resultados sean válidos. También, la prueba de causalidad de `Granger solo evalúa la causalidad en una dirección`; debemos repetir la prueba para verificar que $y_{1,t}$ también causa de Granger a $y_{2,t}$ para que el modelo VAR sea válido. De lo contrario, debemos recurrir al modelo SARIMAX y predecir cada serie temporal por separado.

La hipótesis nula para esta prueba establece que $y_{2,t}$ no causa de Granger a $y_{1,t}$. Nuevamente, utilizaremos el valor p con un valor crítico de 0.05 para determinar si rechazaremos la hipótesis nula o no. En el caso de que el valor p devuelto de la prueba de causalidad de Granger sea menor que 0.05, podemos rechazar la hipótesis nula y decir que $y_{2,t}$ causa de Granger a $y_{1,t}$.